In [1]:
import pandas as pd

df = pd.read_csv("shirts.csv")
print(df)

   height(cm)  weight(kg) t-shirt size
0         158          58            M
1         163          61            M
2         165          61            L
3         168          66            L


# Intro to Machine Learning
* Supervised machine learning: labeled data (i.e., the dataset has an attribute we are interested in predicting for unseen instances)
    * The attribute we want to predict is called the *class* or the *target*
    * If the class is categorical -> classification task
    * If the class is numeric -> regression task
    * Example algorithm: k nearest neighbors (kNN) classifier and regressor
* Unsupervised machine learning: unlabeled data (i.e., the dataset does not have an attribute we are interested in predicting)
    * E.g., clustering/grouping, associations, outliers/anomalies, trends, etc.
    * Example algorithm: k means clustering

## Supervised ML
* We need to divide a dataset into *training* and *testing* sets
    * We train/build an algorithm/model using a training set
    * We evaluate the algorithm/model using a testing set
    * The training and testing sets are *different*

## kNN Example w/Sci-kit Learn

In [2]:
# notation/convention of sci-kit learn API
# X: feature matrix (2D data structure)
# rows are instances and columns are attributes/features
# y: class column (1D data structure)
# the attribute/feature we want to predict
# AKA target array, target vector, labels, etc.
# _train and _test are used to denote training and testing sets, respectively
X_train = df.drop(["t-shirt size"], axis="columns")
y_train = df["t-shirt size"]
print(X_train)
print(y_train)

   height(cm)  weight(kg)
0         158          58
1         163          61
2         165          61
3         168          66
0    M
1    M
2    L
3    L
Name: t-shirt size, dtype: object


In [3]:
# let's scale our feature values
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# typically combine fit() transform() calls fit_transform()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
print(X_train_scaled)
X_test = [[161, 63]] #2D
X_test_scaled = scaler.transform(X_test)
print(X_test_scaled)

[[0.    0.   ]
 [0.5   0.375]
 [0.7   0.375]
 [1.    1.   ]]
[[0.3   0.625]]


In [4]:
# now for kNN!
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
# build/train
knn_clf.fit(X_train_scaled, y_train)
# predict
y_predicted = knn_clf.predict(X_test_scaled)
print(y_predicted)
# challenge: can you print out the neighbor distances?
# read the docs: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

['M']


/Users/sprint/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## Closing Thoughts on kNN
TODO